In [1]:
import tensorflow as tf
import ktrain

if tf.test.gpu_device_name(): 

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

Default GPU Device:/device:GPU:0


In [2]:

import ktrain
from ktrain import text

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

num_folds = 5

run_precision = []
run_recall = []
run_f1score = []
run_accuracy = []

kf = KFold(n_splits=5, shuffle=False, random_state=125)

In [4]:
data = pd.read_csv('sstubsIn_edited.csv')

In [5]:
print(data.shape)

(10231, 5)


In [6]:
#print("pre-processing train data...")

from sklearn.utils import shuffle
from tqdm import tqdm

#label_names = ["bugType"]
#y_train = data[label_names].values

raw_docs_train = data['sourceBeforeFix'].tolist()
#raw_docs_test = test_data['message'].tolist() 
num_classes = 4

#processed_docs_train = []
#for doc in tqdm(raw_docs_train):
#    tokens = word_tokenize(doc)
#    filtered = [word for word in tokens if word not in stop_words]
#    processed_docs_train.append(" ".join(filtered))
#end for
'''
processed_docs_test = []
for doc in tqdm(raw_docs_test):
    tokens = word_tokenize(doc)
    filtered = [word for word in tokens if word not in stop_words]
    processed_docs_test.append(" ".join(filtered))
#end for
'''


#print("tokenizing input data...")
#tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
#tokenizer.fit_on_texts(processed_docs_train )  #leaky
#word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
#word_seq_test = tokenizer.texts_to_sequences(processed_docs_test)
#word_index = tokenizer.word_index
#print("dictionary size: ", len(word_index))

#pad sequences
#word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_train = raw_docs_train
word_seq_train = np.array(word_seq_train)

#data.bugType.value_counts()

In [7]:
class_names = [0,1,2,3]

In [8]:
encoding = {
    'CHANGE_IDENTIFIER': 0, 
    'CHANGE_MODIFIER': 0,
    'CHANGE_NUMERAL': 0, 
    'SWAP_BOOLEAN_LITERAL': 0,
     'DIFFERENT_METHOD_SAME_ARGS': 1, 
     'OVERLOAD_METHOD_MORE_ARGS': 1, 
    'OVERLOAD_METHOD_DELETED_ARGS': 1,
     'CHANGE_CALLER_IN_FUNCTION_CALL': 1, 
     'SWAP_ARGUMENTS': 1,
     'CHANGE_OPERATOR': 2, 
    'CHANGE_UNARY_OPERATOR': 2, 
    'CHANGE_OPERAND': 2, 
     'LESS_SPECIFIC_IF': 3,
    'MORE_SPECIFIC_IF': 3,
     'ADD_THROWS_EXCEPTION': 3, 
      'DELETE_THROWS_EXCEPTION':3,
       
}

In [9]:
y_train_initial= data['bugType'].values


y_train_initial = [encoding[x] for x in y_train_initial]
y_train_initial=np.array(y_train_initial)

In [17]:
print(y_train_initial)

[2 1 1 ... 0 1 0]


In [19]:
unique, counts = np.unique(y_train_initial, return_counts=True)

print(np.asarray((unique, counts)).T)

[[   0 6423]
 [   1 2737]
 [   2  565]
 [   3  506]]


In [13]:
import tensorflow as tf
from sklearn import metrics
from sklearn.model_selection import train_test_split
count = 1

for train_index, test_index in kf.split(word_seq_train):
    x_trn, x_tst = word_seq_train[train_index], word_seq_train[test_index]
    y_trn, y_tst = y_train_initial[train_index], y_train_initial[test_index]
    
    x_new_train, x_val, y_new_train, y_val= train_test_split(x_trn, y_trn, test_size=0.11115, random_state=125)
    
    print("\nFold ", count)
    
    (x_train,  y_train), (x_test, y_test),  preproc = text.texts_from_array(x_train=x_new_train, y_train=y_new_train,
                                                                      x_test=x_tst, y_test=y_tst,
                                                                       class_names=class_names,
                                                                       preprocess_mode='standard',
                                                                       maxlen=150, 
                                                                       max_features=1000)
    
    model = text.text_classifier('nbsvm', train_data=(x_train, y_train), preproc=preproc)
    
    learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=10)
    
    
    callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)
    #learner.lr_find()

    
    #learner.fit_onecycle(2e-5, 40, callbacks=[callbacks])

    learner.autofit(0.01)

    
    #predictor = ktrain.get_predictor(learner.model, preproc)
    #predictor.get_classes()
    
    (x_train,  y_train), (x_test, y_test),  preproc = text.texts_from_array(x_train=x_val, y_train=y_val,
                                                                       x_test=x_val, y_test=y_val,
                                                                       
                                                                       class_names=class_names,
                                                                       preprocess_mode='standard',
                                                                       maxlen=150, 
                                                                       max_features=1000)
    
    learner.validate(val_data=(x_test, y_test), class_names=class_names)
    
    '''
    print(metrics.classification_report(y_tst, y_pred))
    
    precision = precision_score(y_tst, y_pred, pos_label=1)
    recall = recall_score(y_tst, y_pred, pos_label=1)
    f1score = f1_score(y_tst, y_pred, pos_label=1)
    lstm_accuracy = accuracy_score(y_tst, y_pred)

    run_accuracy.append(accuracy)
    run_f1score.append(f1score)
    run_precision.append(precision)
    run_recall.append(recall)
    '''
    count = count+1
    #X_train = data['sourceBeforeFix'].tolist()


Fold  1
language: en
Word Counts: 7750
Nrows: 7274
7274 train sequences
train sequence lengths:
	mean : 5
	95percentile : 12
	99percentile : 45
x_train shape: (7274,150)
y_train shape: (7274, 4)
Is Multi-Label? False
2047 test sequences
test sequence lengths:
	mean : 2
	95percentile : 7
	99percentile : 15
x_test shape: (2047,150)
y_test shape: (2047, 4)
task: text classification
Is Multi-Label? False
compiling word ID features...
maxlen is 150
building document-term matrix... this may take a few moments...
rows: 1-7274
computing log-count ratios...
done.
early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 0.01...
Epoch 1/1024
728/728 [==============================] - 3s 3ms/step - loss: 1.1943 - accuracy: 0.4983 - val_loss: 0.9597 - val_accuracy: 0.6072
Epoch 2/1024
728/728 [==============================] - 2s 3ms/step - loss: 0.9218 - accuracy: 0.6080 - val_los


Fold  4
language: en
Word Counts: 7130
Nrows: 7275
7275 train sequences
train sequence lengths:
	mean : 5
	95percentile : 12
	99percentile : 41
x_train shape: (7275,150)
y_train shape: (7275, 4)
Is Multi-Label? False
2046 test sequences
test sequence lengths:
	mean : 4
	95percentile : 7
	99percentile : 26
x_test shape: (2046,150)
y_test shape: (2046, 4)
task: text classification
Is Multi-Label? False
compiling word ID features...
maxlen is 150
building document-term matrix... this may take a few moments...
rows: 1-7275
computing log-count ratios...
done.
early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 0.01...
Epoch 1/1024
728/728 [==============================] - 2s 3ms/step - loss: 1.1691 - accuracy: 0.5215 - val_loss: 1.1103 - val_accuracy: 0.5411
Epoch 2/1024
728/728 [==============================] - 3s 4ms/step - loss: 0.9150 - accuracy: 0.6079 - val_los